<a href="https://colab.research.google.com/github/AshbeeKim/dadaiksunTeamProject/blob/main/ipynb/AshbeeKim/KMA_api_list_Dynamic_WebC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Access to Google Drive for co-working

* Purpose : 목적에 맞는 기상청 api 자료를 확인

[나중에 읽어볼 웹크롤링 관련 링크]https://velog.io/@mowinckel/%EC%9B%B9-%ED%81%AC%EB%A1%A4%EB%A7%81-I

In [ ]:
import os

from google.colab import drive  
# 구글 드라이브 접근
ROOT = "/content/drive"
drive.mount(ROOT)

from os.path import join
# 개인 폴더
PROJECT = "MyDrive/Task/Proj_WeatherGo"
PROJ_PATH = join(ROOT, PROJECT)

CHK_PATH = join(PROJ_PATH, "MINE")

# 공유 폴더
CO_WORK = "MyDrive/Proj_WT"
CW_PATH = join(ROOT, CO_WORK)

# CODES = "Codes"     # 코드/마크다운 공유 및 에러 해결 목적
# CODE_PATH = join(CW_PATH, CODES)
# # !mkdir "{CODE_PATH}"

PAPERS = "Papers"   # 논문 공유 폴더
PAPER_PATH = join(CW_PATH, PAPERS)
# !mkdir "{PAPER_PATH}"

DATAS = "DataSets"  # 데이터 공유 폴더
DATA_PATH = join(CW_PATH, DATAS)
# !mkdir "{DATA_PATH}"

DA = "Analysis"     # 분석 공유 폴더
DA_PATH = join(CW_PATH, DA)
# !mkdir "{DA_PATH}"

Mounted at /content/drive


# Baseline

## install packages

In [ ]:
#파이썬 위치
!which python

#파이썬 버전
!python -V

!which pip

# colab에 기본적으로 깔려있는 프로그램
# !ls /usr/lib/

/usr/local/bin/python
Python 3.7.10
/usr/local/bin/pip


In [ ]:
# Base
import sys
import re

if 'google.colab' in sys.modules:
  !apt-get update
  !apt install chromium-chromedriver
  !cp /usr/lib/chromium-browser/chromedriver /usr/bin
  !pip install selenium
  !pip install xmltodict

import time
import random
import math
import ast
from ast import literal_eval

if 'ipykernel' in sys.modules:
    from tqdm.notebook import tqdm
else:
    from tqdm import tqdm

# DataFrame and Visualization
import csv
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
from IPython.core.display import display, HTML

# Web Scraping(Web Crawling)
import json
import requests
from bs4 import BeautifulSoup as BS

import urllib
import urllib.request
from urllib.request import urlopen
from urllib.parse import urlencode, unquote, quote_plus
from urllib.error import HTTPError, URLError

import xmltodict
import xml.dom.minidom
from xml.etree.ElementTree import parse

import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

# KMA(한국기상청)_정적크롤링(requests, BeautifulSoup)

## Main Page

[link] https://www.weather.go.kr/w/index.do

In [ ]:
mainHP = "https://www.weather.go.kr/w/index.do"

데이터 품질 현황을 보면, 종관기상관측 데이터가 방재기상관측 데이터 대비 정확도율이 높음(거의 100%에 수렴함)

## Data Page

In [ ]:
DataP = "https://data.kma.go.kr/data/grnd/selectAsosRltmList.do?pgmNo=36"

In [ ]:
req = requests.get(DataP).text

In [ ]:
soup = BS(req, "html.parser") #lxml

### Data Titles

In [ ]:
title = []
titles = soup.find_all("a", {"class":"d1 ico_aft"})
for t in titles:
  title.append(t.text)

### initialize list and DF using Titles

In [ ]:
#List
for t_lst in title:
  locals()[f"lst_{t_lst}"] = []

#DataFrame
for t_df in title:
  locals()[f"df_{t_df}"] = pd.DataFrame()

### Data List

우리가 사용하고자 하는 데이터는 예측값이 아닌 관측값으로 확인하는 것이기 때문에 우선 기상관측 데이터 목록만 불러옴

In [ ]:
#기상관측 : 5개의 리스트

for child in range(1, 6):
  lst = soup.select_one(f"#snb > nav > ul > li.it_d1.on > ul > li:nth-of-type({child}) > a.d2").text
  lst_기상관측.append(lst)

# nth-child -> nth-of-type
#snb > nav > ul > li.it_d1.on > ul > li:nth-child(1) > a.d2

In [ ]:
lst_기상관측

['지상', '해양', '고층', '항공', '세계기상전문(GTS)']

In [ ]:
#List
for idx, l_lst in enumerate(lst_기상관측):
  locals()[f"lst_{l_lst}"] = []
  lists_a = soup.select(f"#snb > nav > ul > li.it_d1.on > ul > li:nth-of-type({idx+1}) > ul > li")
  for index in range(1, len(lists_a)+1):
    lst = soup.select_one(f"#snb > nav > ul > li.it_d1.on > ul > li:nth-of-type({idx+1}) > ul > li:nth-of-type({index}) > a").text
    # soup.select로만 하면 list의 형태로 입력받아서 .text로 안의 글자만 불러 올 수 없음
    locals()[f"lst_{l_lst}"].append(lst)
  print(f"'{l_lst} 기상 데이터'의 종류로는",locals()[f"lst_{l_lst}"],"가 있습니다.\n")
  print("="*200)

#snb > nav > ul > li.it_d1.on > ul > li.it_d2.on > ul > li.on > a #지상 list에서 들어간 기상관측종류
#snb > nav > ul > li.it_d1.on > ul > li.it_d2.hv > ul > li:nth-child(2) > a #해양 list에서 들어간 기상관측종류
#snb > nav > ul > li.it_d1.on > ul > li:nth-child(2) > ul > li:nth-child(2) > a

'지상 기상 데이터'의 종류로는 ['종관기상관측(ASOS)', '방재기상관측(AWS)', '농업기상관측(AAOS)', '북한기상관측', '공공기관 기상관측', '계절관측', '황사관측(PM10)', '낙뢰관측'] 가 있습니다.

'해양 기상 데이터'의 종류로는 ['해양기상부이', '등표기상관측', '파고부이'] 가 있습니다.

'고층 기상 데이터'의 종류로는 ['레윈존데', '연직바람관측'] 가 있습니다.

'항공 기상 데이터'의 종류로는 ['공항기상관측(AMOS)'] 가 있습니다.

'세계기상전문(GTS) 기상 데이터'의 종류로는 ['지상(SYNOP)', '부이(BUOY)', '고층(TEMP)', '태풍예보(TYP)', '위험기상정보(SIGMET)', '기후(CLIMAT)'] 가 있습니다.



# KMA(한국기상청)_동적크롤링(selenium, json)

## Colab WebC Setup

In [ ]:
c_options = webdriver.ChromeOptions ()
c_options.add_argument ('--headless')
c_options.add_argument ('--no-sandbox')
c_options.add_argument ('--disable-dev-shm-usage')

# wd = webdriver.Chrome('/content/chromedriver', options=c_options)
# wd = webdriver.Chrome('/usr/bin/chromedriver', options=c_options)

## Api List Page

api가 있다면 쉽게 활용하는 편이 훨씬 편하지 않을까?

In [ ]:
wd = webdriver.Chrome('chromedriver', options=c_options)

In [ ]:
kmaapiP = "https://data.kma.go.kr/api/selectApiList.do"

In [ ]:
wd.get(kmaapiP)
wd.implicitly_wait(random.randint(2,6)) #give some time sleep randomly 2 secs to 6secs
soup = BS(wd.page_source, "html.parser")
print(f"Accessed URL : [{kmaapiP}]")

Accessed URL : [https://data.kma.go.kr/api/selectApiList.do]


In [ ]:
#페이지에서 주석 보는뎈ㅋㅋㅋㅋ겁나 웃김
  #"<!--디자이너가 주다 말았네....-->"에서 현웃 터짐ㅋㅋㅋㅋㅋ 

total_cnt = wd.find_element_by_css_selector("#schForm > div > div.total > b").text.split(" ")[1]
print(f"활용 가능한 전체 api의 수 : {total_cnt}")
wd.implicitly_wait(random.randint(2,6))

활용 가능한 전체 api의 수 : 34건


In [ ]:
lst_pages = wd.find_element_by_css_selector("#wrap_content > div.wrap_itm.board > div.ft_lst > div > ul").text.split('\n')
# 단, string 형태로 저장됨

wd.implicitly_wait(random.randint(2,6))

['1', '2', '3', '4'] <class 'str'>


In [ ]:


#wrap_content > div.wrap_itm.board > div.ft_lst > div > ul > li.on > span

In [ ]:
wd.close()
print(f"Closed URL : [{kmaapiP}]")
# Once closed URL page, should re-run "webdriver.Chrome('chromedriver', options=c_options)"

Closed URL : [https://data.kma.go.kr/api/selectApiList.do]


In [ ]:
def saveToFile(filename, df):
  with open()

로컬로 크롤링 하는 것과는 다르게, 새창 혹은 새탭으로 열리지 않음.

그래도 html.parser로 스크립트 보면서 id나 class 확인이 가능하고, 해당 사이트에서 x_path를 가져오는 방법도 있을 것임.

방대한 데이터를 바로 드라이브에 저장하려면, colab에서 사용하는 게 나을텐데,,, input에 따라서 달라지는 결과나, 사이트 레이아웃 및 구조가 변경되는 경우를 생각하면 로컬에서 구동하는 게 나을 수도 있음